<a href="https://colab.research.google.com/github/sambitmishra98/PyFR-ideal-performance/blob/main/performance_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projected performance computation from mesh

In this document, we aim to compare the performance of expected PyFR performance in comparison with actual performance.

The expected performance is computed by understanding inputs and outputs to all kernels used for computation.

The actual performance is computed using `perf_counter()` and looking at wall-time in the solution file. For example, actual performance of Intel MAX GPUs is shown in a [Google Docs file](https://docs.google.com/document/d/1yX7JqTTsXRikTtzRon-03TgRGceByce075N4-Ptp7cI/edit?usp=sharing) (Restricted access). The latter method was used to benchmark performance of PyFR on A100 GPUs in a paper: [Scaling Study of Flow Simulations on Composable Cyberinfrastructure](https://doi.org/10.1145/3569951.3597565).

In [2]:
# Global variables

size_of_float = 4 
precision_size = {'single': size_of_float, 
                  'double': 2*size_of_float}

etypes = ['tet', 'pyr', 'pri', 'hex',]
element_counts = {etype: 0 for etype in etypes}


## Performance details from configuration file

Following the configurations as given in [PyFR documentation](https://pyfr.readthedocs.io/en/latest/user_guide.html#configuration-file-ini).
Only those relevant to performance computation is declared below.


In [3]:
# [backend]
precision = 'double'

# [solver]
system = 'navier-stokes'
order = 1

# [solver-time-integrator]
scheme = 'rk4'
tstart = 0
tend = 1.0001
dt = 0.0001

## Processing data from mesh


Details of how mesh size may be obtained is found in plugin path `pyfr/plugins/benchmark.py` in benchmark branch in [sambitmihsra98/PyFR.git](https://github.com/sambitmishra98/PyFR.git).


In [4]:
ndims = 3
nvars = ndims + 2

element_counts['tet'] = 1

## Degrees of Freedom (DoFs) calculation

An analysis of Flux Reconstruction schemes on Tetrahedral elements may be found in [this paper](https://doi.org/10.1007/s10915-016-0204-y). In general, we have ...

In [5]:
def edof(etype, n):
    if etype == 'tri':
        Nu = (n+1)*(n+2)/2
        Nf = 3*(n+1)
    elif etype == 'quad':
        Nu = (n+1)**2
        Nf = 4*(n+1)
    elif etype == 'tet':
        Nu = (n+3)*edof('tri', n)[0]//3
        Nf = 4*edof('tri', n)[0]
    elif etype == 'pyr':
        Nu = (2*n+3)*edof('tri', n)[0]//3
        Nf = 4*edof('tri', n)[0] +   edof('quad', n)[0]
    elif etype == 'pri':
        Nu = (n+1)*edof('tri', n)[0]
        Nf = 2*edof('tri', n)[0] + 3*edof('quad', n)[0]
    elif etype == 'hex':
        Nu = (n+1)*edof('quad', n)[0]
        Nf = 6*edof('quad', n)[0]
    else:
        raise Exception("Not implemented yet")

    NNuf = Nu*Nf    # Matrix of size Nu x Nf
    NNuu = Nu*Nu    # Matrix of size Nu x Nu

    return int(Nu), int(Nf), int(NNuf), int(NNuu)


In [6]:
# Get total number of degrees of freedom on the basis of the element counts and the element type
dofs_s = sum(n*edof(etype, order)[0] for etype, n in element_counts.items())
dofs_f = sum(n*edof(etype, order)[1] for etype, n in element_counts.items())
M_sf   = sum(n*edof(etype, order)[2] for etype, n in element_counts.items())
M_ss   = sum(n*edof(etype, order)[3] for etype, n in element_counts.items())


## Storage

Size of registers for explicit RK stages is as per `stepper_nregs` found in `pyfr/integrators/std/steppers.py`.

In [7]:
rk_registers = {'euler': 2, 
                'rk4'  : 3,}

In [8]:
# Storage values in bytes
solution_storages = {
       'scalar u': precision_size[precision]*nvars*dofs_s*rk_registers[scheme],
       'vector u': precision_size[precision]*nvars*dofs_s*ndims,
     'S matrix u': precision_size[precision]*dofs_s*ndims**2,
        '1/|J| u': precision_size[precision]*dofs_s,
                    }     

# Flux
flux_storages = {
     'scalar f': precision_size[precision]*nvars*dofs_f,
     'vector f': precision_size[precision]*nvars*dofs_f*ndims if system == 'navier-stokes' else 0,
}
# Interface-normal storages
interface_normal_storages = {
           'n/|n| i': precision_size[precision]*dofs_f*ndims/2,
             '|n| i': precision_size[precision]*dofs_f,
     'scalar view i': precision_size[ 'single']*dofs_f*2,
     'vector view i': precision_size[ 'single']*dofs_f*ndims if system == 'navier-stokes' else 0,
}

storages = solution_storages|flux_storages|interface_normal_storages
from pprint import pprint
storages['total'] = sum(storages.values())
pprint(storages)

{'1/|J| u': 32,
 'S matrix u': 288,
 'n/|n| i': 144.0,
 'scalar f': 480,
 'scalar u': 480,
 'scalar view i': 96,
 'total': 3680.0,
 'vector f': 1440,
 'vector u': 480,
 'vector view i': 144,
 '|n| i': 96}


## Over one sweep of the integrator ...
Total number of computations performed is a function of kernels.
The inputs, outputs and computations in each kernel needs to be understood.

### ... total number of floating point operations performed 

In [9]:
# Store dictionary of computations, inputs and outputs

soln_flux_matrix_computations = sum([n*edof(k,order)[0]*edof(k,order)[1] for k, n in element_counts.items()])*2*nvars
soln_soln_matrix_computations = sum([n*edof(k,order)[0]*edof(k,order)[0] for k, n in element_counts.items()])*2*nvars*ndims

if ndims == 2:
    if system == 'euler':
        non_Ms = {'Gradcoru':         0, 'Tflux':  44*dofs_s, 'Rsolves':  92*dofs_f/2,}
    else:
        non_Ms = {'Gradcoru': 32*dofs_f, 'Tflux':  91*dofs_s, 'Rsolves': 200*dofs_f/2,}
elif ndims == 3:
    if system == 'euler':
        non_Ms = {'Gradcoru':         0, 'Tflux': 105*dofs_s, 'Rsolves': 140*dofs_f/2,}
    else:
        non_Ms = {'Gradcoru': 90*dofs_s, 'Tflux': 189*dofs_s, 'Rsolves': 269*dofs_f/2,}
#                                                         https://github.com/sambitmishra98/PyFR/blob/benchmark/pyfr/.................................
# Computations
Ms = {'M0'  : soln_flux_matrix_computations,                                            # disu,            u*f      solvers/baseadvec/elements.py#L73
      'M132': soln_soln_matrix_computations,                                            # qptsu,           dims*u*u solvers/baseadvec/elements.py#L90
      'M3'  : soln_flux_matrix_computations,                                            # tdivtpcorf,      u*f      solvers/baseadvec/elements.py#L97
      'M460': soln_soln_matrix_computations       if system == 'navier-stokes' else 0,  # tgradpcoru_upts, u*u      solvers/baseadvecdiff/elements.py#L34 
      'M6'  : soln_flux_matrix_computations*ndims if system == 'navier-stokes' else 0,  # tgradcoru_upts,  dims*u*f solvers/baseadvecdiff/elements.py#L38
      'M5'  : soln_flux_matrix_computations*ndims if system == 'navier-stokes' else 0,  # mul,             dims*u*f solvers/baseadvecdiff/elements.py#L68
}

others={'Conu'    : 0,
        'Rcpdjac' : nvars*dofs_s, 
}

kernels = Ms|non_Ms|others


### ... total reads and writes performed

Matrix multiplications are the costliest parts. Hence, vector multiplications shall be ignored and only M-related computations shall be considered.


In [10]:
#def edof(etype, n):
#    if etype == 'tri':
#        Nu = (n+1)*(n+2)/2
#        Nf = 3*(n+1)
#    elif etype == 'quad':
#        Nu = (n+1)**2
#        Nf = 4*(n+1)
#    elif etype == 'tet':
#        Nu = (n+3)*edof('tri', n)[0]//3
#        Nf = 4*edof('tri', n)[0]
#    elif etype == 'pyr':
#        Nu = (2*n+3)*edof('tri', n)[0]//3
#        Nf = 4*edof('tri', n)[0] +   edof('quad', n)[0]
#    elif etype == 'pri':
#        Nu = (n+1)*edof('tri', n)[0]
#        Nf = 2*edof('tri', n)[0] + 3*edof('quad', n)[0]
#    elif etype == 'hex':
#        Nu = (n+1)*edof('quad', n)[0]
#        Nf = 6*edof('quad', n)[0]
#    else:
#        raise Exception("Not implemented yet")
#
#    NNuf = Nu*Nf    # Matrix of size Nu x Nf
#    NNuu = Nu*Nu    # Matrix of size Nu x Nu
#
#    return int(Nu), int(Nf), int(NNuf), int(NNuu)
#
#dofs_s = sum(n*edof(etype, order)[0] for etype, n in element_counts.items())
#dofs_f = sum(n*edof(etype, order)[1] for etype, n in element_counts.items())
#
#
#M_sf   = sum(n*edof(etype, order)[2] for etype, n in element_counts.items())
#M_ss   = sum(n*edof(etype, order)[3] for etype, n in element_counts.items())

#soln_flux_matrix_computations = sum([n*edof(k,order)[0]*edof(k,order)[1] for k, n in element_counts.items()])*2*nvars
#soln_soln_matrix_computations = sum([n*edof(k,order)[0]*edof(k,order)[0] for k, n in element_counts.items()])*2*nvars*ndims
## Computations
#Ms = {'M0'  : soln_flux_matrix_computations,                                            # disu,            u*f      solvers/baseadvec/elements.py#L73
#      'M132': soln_soln_matrix_computations,                                            # qptsu,           dims*u*u solvers/baseadvec/elements.py#L90
#      'M3'  : soln_flux_matrix_computations,                                            # tdivtpcorf,      u*f      solvers/baseadvec/elements.py#L97
#      'M460': soln_soln_matrix_computations       if system == 'navier-stokes' else 0,  # tgradpcoru_upts, u*u      solvers/baseadvecdiff/elements.py#L34 
#      'M6'  : soln_flux_matrix_computations*ndims if system == 'navier-stokes' else 0,  # tgradcoru_upts,  dims*u*f solvers/baseadvecdiff/elements.py#L38
#      'M5'  : soln_flux_matrix_computations*ndims if system == 'navier-stokes' else 0,  # mul,             dims*u*f solvers/baseadvecdiff/elements.py#L68
#}

vector_reads  = {
    'M0'  :       dofs_s,
    'M132': ndims*dofs_s,
    'M3'  :       dofs_f,
    'M460':       dofs_s,
    'M6'  : ndims*dofs_f,
    'M5'  : ndims*dofs_f,
    }

cached_matrix_reads = {
    'M0'  :       M_sf,
    'M132': ndims*M_ss,
    'M3'  :       M_sf,
    'M460':       M_ss,
    'M6'  : ndims*M_sf,
    'M5'  : ndims*M_sf,
    }

vector_writes = {
    'M0'  :       dofs_f,
    'M132':       dofs_s,
    'M3'  :       dofs_s,
    'M460':       dofs_s,
    'M6'  :       dofs_s,
    'M5'  :       dofs_s,
    }

communication_per_timestep = max(sum(vector_reads.values()) + sum(cached_matrix_reads.values()), sum(vector_writes.values()))

pprint(vector_reads, width=1)
pprint(cached_matrix_reads, width=1)
pprint(vector_writes, width=1)
pprint(communication_per_timestep, width=1)

{'M0': 4,
 'M132': 12,
 'M3': 12,
 'M460': 4,
 'M5': 36,
 'M6': 36}
{'M0': 48,
 'M132': 48,
 'M3': 48,
 'M460': 16,
 'M5': 144,
 'M6': 144}
{'M0': 12,
 'M132': 4,
 'M3': 4,
 'M460': 4,
 'M5': 4,
 'M6': 4}
552
